<a href="https://colab.research.google.com/github/poojakota17/Data-Mining-255/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use state of art libraries for Approximate nearest neighbor search for a  data set

> Dataset : https://making.lyst.com/lightfm/docs/_modules/lightfm/datasets/stackexchange.html

> It is a stackexchange dataset consists of user-question interaction and question and tags item features

## Install the required libraries

In [1]:
!pip install lightFM

     |████████████████████████████████| 310 kB 4.9 MB/s 
  Created wheel for lightFM: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705362 sha256=59079be29e68bdb8cbf73cc4a3b2dbb6b67e7c19e8c674abd418f539f6c53b4c
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightFM


In [10]:
!pip install faiss

In [11]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [13]:
!python3 -m pip install --upgrade faiss-gpu==1.7.1

     |████████████████████████████████| 89.7 MB 16 kB/s 
  Attempting uninstall: faiss-gpu
    Found existing installation: faiss-gpu 1.7.1.post2
    Uninstalling faiss-gpu-1.7.1.post2:
      Successfully uninstalled faiss-gpu-1.7.1.post2


In [14]:
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import pickle
import faiss

### Retrieve the dataset using fetch_stackexchange() and model it using LightFM to create a vector embedding of item and features.

In [133]:
stackexchange = fetch_stackexchange('crossvalidated',indicator_features=True,tag_features=True)
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

> The item_features looks like this. (0,72360) 1.0, means that for question id 0, 72360 is the tag id(feature id) and the data here is 1( since its a sparse matrix)

In [134]:
print(stackexchange['item_features'])

  (0, 0)	1.0
  (0, 72360)	1.0
  (0, 72361)	1.0
  (0, 72362)	1.0
  (1, 1)	1.0
  (1, 72363)	1.0
  (1, 72364)	1.0
  (2, 2)	1.0
  (2, 72365)	1.0
  (2, 72366)	1.0
  (3, 3)	1.0
  (3, 72363)	1.0
  (3, 72367)	1.0
  (4, 4)	1.0
  (4, 72368)	1.0
  (5, 5)	1.0
  (5, 72369)	1.0
  (5, 72370)	1.0
  (5, 72371)	1.0
  (5, 72372)	1.0
  (6, 6)	1.0
  (6, 72373)	1.0
  (7, 7)	1.0
  (7, 72374)	1.0
  (7, 72375)	1.0
  :	:
  (72354, 72837)	1.0
  (72354, 73124)	1.0
  (72354, 73164)	1.0
  (72355, 72355)	1.0
  (72355, 72436)	1.0
  (72355, 72548)	1.0
  (72355, 73090)	1.0
  (72356, 72356)	1.0
  (72356, 72440)	1.0
  (72356, 72513)	1.0
  (72356, 72731)	1.0
  (72356, 72796)	1.0
  (72356, 73057)	1.0
  (72357, 72357)	1.0
  (72357, 72404)	1.0
  (72357, 73399)	1.0
  (72358, 72358)	1.0
  (72358, 72406)	1.0
  (72358, 72411)	1.0
  (72358, 72747)	1.0
  (72358, 72920)	1.0
  (72359, 72359)	1.0
  (72359, 72503)	1.0
  (72359, 72507)	1.0
  (72359, 72616)	1.0


Open a pickle file in writing mode and store the feature labels, vector and item features, so that it can be used again without re-loading 

In [101]:
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors,"features":stackexchange['item_features']}, f)

In [173]:
print(stackexchange['item_feature_labels'])

['question_id:0' 'question_id:1' 'question_id:2' ... 'events'
 'mutlivariate' 'sample-variance']


> Load the data from the pickle file

In [71]:
import pickle
def load_data():
    with open('stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data
data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ..., 'events',
        'mutlivariate', 'sample-variance'], dtype='<U50'),
 'vector': array([[ 0.16250771,  0.12406151, -0.08036248, ..., -0.11496134,
         -0.02402518, -0.00704376],
        [ 0.02717981,  0.25576022, -0.204706  , ..., -0.39978805,
          0.15229318,  0.08870568],
        [ 0.03993656,  0.03734043, -0.1159954 , ..., -0.30582762,
         -0.12722601, -0.09749138],
        ...,
        [ 0.01483956,  0.05262152,  0.09099412, ...,  0.11744755,
         -0.00935212,  0.000483  ],
        [-0.11480884,  0.03977454,  0.11133081, ..., -0.04319084,
          0.05634591,  0.16093293],
        [ 0.05669373, -0.30438346, -0.00119976, ..., -0.12383866,
         -0.09578706,  0.08885497]], dtype=float32)}

### Function to retrieve the tags of a question based on question id

> As an eg, for question id :0, these are the tags : ['bayesian', 'prior', 'elicitation']

In [124]:
import scipy.sparse
import random
import itertools
cx = scipy.sparse.coo_matrix(stackexchange['item_features'])
def using_coo(index):
    arr=[]
       
    for i,j,v in zip(cx.row, cx.col, cx.data):
      if i==index and i != j:
        arr.append(data['name'][j])
      elif i>index:
        break
    return arr
      # print(j,data['name'][j])
using_coo(0)

['bayesian', 'prior', 'elicitation']

# 1. Locality Sensitive Hashing:

In [199]:
class LSH():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def lsh_build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, self.dimention, number_of_partition, search_in_x_partitions, subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)

      # to get similar tagged questions     
    def get_similar_tagged_questions(self, vectors, k=12):
        distances, indices = self.index.search(vectors, k) 
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
        

> Create an object of class lsh and call the get_similar_tagged_questions function to get similar tags for a particular vector. We can see the ouput below. The output shows 12 question id's which are similar to the input and their corresponding tags

In [200]:
index = LSH(data["vector"], data["name"])
index.lsh_build()
index.get_similar_tagged_questions(data["vector"][0:2])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:10895': ['bayesian',
  'frequentist',
  'hypothesis-testing',
  'confidence-interval',
  'credible-interval'],
 'question_id:1409': ['bayesian', 'prior', 'probability', 'meta-analysis'],
 'question_id:14098': ['bayesian', 'nonparametric-bayes'],
 'question_id:14480': ['bayesian', 'replication'],
 'question_id:2021': ['bayesian', 'prior', 'terminology'],
 'question_id:27035': ['bayesian', 'statistical-significance', 'ab-test'],
 'question_id:3063': ['bayesian', 'multilevel-analysis'],
 'question_id:3102': ['bayesian', 'multilevel-analysis', 'identifiability'],
 'question_id:40715': ['bayesian',
  'prior',
  'estimation',
  'parametric',
  'uninformative-prior'],
 'question_id:45542': ['bayesian', 'prior', 'elicitation'],
 'question_id:9365': ['bayesian',
  'prior',
  'likelihood-function',
  'inference',
  'information']}

# 2. Exhaustive Search

In [201]:
class ExhaustiveSearch():

    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(self.vectors)
        
    # to get similar tagged questions      
    def get_similar_tagged_questions(self,vectors,k=12):
        distances, indices = self.index.search(vectors, k) 
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
       

> Create an object of class ExhaustiveSearch and call the get_similar_tagged_questions function to get similar tags for a particular vector. We can see the ouput below. The output shows 12 question id's which are similar to the input and their corresponding tags

In [202]:
index = ExhaustiveSearch(data["vector"], data["name"])
index.get_similar_tagged_questions(data['vector'][0:2])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:14826': ['bayesian', 'elicitation', 'r'],
 'question_id:17409': ['bayesian', 'modeling', 'hierarchical-bayesian'],
 'question_id:29382': ['bayesian',
  'prior',
  'elicitation',
  'regression',
  'hyperparameter'],
 'question_id:29552': ['bayesian', 'prior', 'elicitation'],
 'question_id:32951': ['bayesian', 'prior'],
 'question_id:45542': ['bayesian', 'prior', 'elicitation'],
 'question_id:53448': ['bayesian', 'prior'],
 'question_id:60904': ['bayesian', 'prior'],
 'question_id:62277': ['bayesian', 'prior', 'optimization'],
 'question_id:65706': ['bayesian', 'prior'],
 'question_id:71861': ['bayesian', 'prior']}

# 3. Product Quantization

In [197]:
class Prod_quant():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def prod_quant_build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
    # to get similar tagged questions    
    def get_similar_tagged_questions(self, vectors, k=12):
        distances, indices = self.index.search(vectors, k) 
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
        

> Create an object of class Prod_quant and call the get_similar_tagged_questions function to get similar tags for a particular vector. We can see the ouput below. The output shows 12 question id's which are similar to the input and their corresponding tags

In [198]:
index = Prod_quant(data["vector"], data["name"])
index.prod_quant_build()
index.get_similar_tagged_questions(data["vector"][0:2])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:10895': ['bayesian',
  'frequentist',
  'hypothesis-testing',
  'confidence-interval',
  'credible-interval'],
 'question_id:1409': ['bayesian', 'prior', 'probability', 'meta-analysis'],
 'question_id:14098': ['bayesian', 'nonparametric-bayes'],
 'question_id:14480': ['bayesian', 'replication'],
 'question_id:2021': ['bayesian', 'prior', 'terminology'],
 'question_id:27035': ['bayesian', 'statistical-significance', 'ab-test'],
 'question_id:3063': ['bayesian', 'multilevel-analysis'],
 'question_id:3102': ['bayesian', 'multilevel-analysis', 'identifiability'],
 'question_id:40715': ['bayesian',
  'prior',
  'estimation',
  'parametric',
  'uninformative-prior'],
 'question_id:45542': ['bayesian', 'prior', 'elicitation'],
 'question_id:9365': ['bayesian',
  'prior',
  'likelihood-function',
  'inference',
  'information']}

# 4. Trees and Graphs

In [161]:
!pip install annoy

     |████████████████████████████████| 646 kB 5.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391679 sha256=891c36d544f0a63892d40d917d5735ea29a124c59b500383e4b742050b2d59c9
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [194]:
import annoy
class Tree_graph_index():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def tree_build(self, number_of_trees=4):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)

      # to get similar tagged questions     
    def get_similar_tagged_questions(self, vector, k=12):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        # print(indices)
        similar_q_w_feature={}
        q_features=[]
        for i in indices:
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
       

> Create an object of class Tree_graph_index and call the get_similar_tagged_questions function to get similar tags for a particular vector. We can see the ouput below. The output shows 12 question id's which are similar to the input and their corresponding tags

In [195]:
index = Tree_graph_index(data["vector"], data["name"])
index.tree_build()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # Remove the CWD from sys.path while we load stuff.


In [196]:
index.get_similar_tagged_questions(data["vector"][0])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:10471': ['mathematics'],
 'question_id:13610': ['statistical-significance',
  'hypothesis-testing',
  't-test',
  'multilevel-analysis'],
 'question_id:17619': ['anova', 'interpretation'],
 'question_id:21533': ['bayesian', 'topic-models'],
 'question_id:22940': ['bayesian', 'modeling', 'hypothesis-testing'],
 'question_id:22970': ['bayesian', 'likelihood-function'],
 'question_id:29878': ['bayesian',
  'hypothesis-testing',
  'categorical-data',
  'bayes'],
 'question_id:37539': ['t-test', 'power'],
 'question_id:53444': ['bayesian', 'prior', 'hypothesis-testing', 'python'],
 'question_id:59680': ['bayesian',
  'hypothesis-testing',
  'hierarchical-bayesian'],
 'question_id:59983': ['bayesian', 'mathematics']}

# 5. HNSW

In [187]:
!pip install  nmslib

     |████████████████████████████████| 13.5 MB 5.1 MB/s 
     |████████████████████████████████| 188 kB 47.8 MB/s 


In [213]:
import nmslib
class hnsw_index():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def hnsw_build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})

      # to get similar tagged questions     
    def get_similar_tagged_questions(self, vector, k=12):
        indices = self.index.knnQuery(vector, k=k)
        similar_q_w_feature={}
        q_features=[]
        for i in indices[0]:
          # similar_q_w_feature.append((self.labels[i], using_coo(i)))
          similar_q_w_feature[self.labels[i]]=using_coo(i)
        return similar_q_w_feature
       

> Create an object of class hnsw_index and call the get_similar_tagged_questions function to get similar tags for a particular vector. We can see the ouput below. The output shows 12 question id's which are similar to the input and their corresponding tags

In [214]:
index = hnsw_index(data["vector"], data["name"])
index.hnsw_build()

In [215]:
index.get_similar_tagged_questions(data["vector"][0])

{'question_id:0': ['bayesian', 'prior', 'elicitation'],
 'question_id:14826': ['bayesian', 'elicitation', 'r'],
 'question_id:22211': ['bayesian', 'power'],
 'question_id:29382': ['bayesian',
  'prior',
  'elicitation',
  'regression',
  'hyperparameter'],
 'question_id:29552': ['bayesian', 'prior', 'elicitation'],
 'question_id:32673': ['bayesian', 'modeling'],
 'question_id:45542': ['bayesian', 'prior', 'elicitation'],
 'question_id:53448': ['bayesian', 'prior'],
 'question_id:60904': ['bayesian', 'prior'],
 'question_id:62277': ['bayesian', 'prior', 'optimization'],
 'question_id:65706': ['bayesian', 'prior'],
 'question_id:71861': ['bayesian', 'prior']}

## Conclusion

> HNSW and Exhaustive search have a similar output.

> Product Quantization and LSH have a similar output.

> Tree and Graphs output is different from others.